In [1]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline


# Chatbot: Training, Inference (ONNX, TORCH)
**Author:** [Phalt97](https://github.com/phalt97)


In this tutorial, we explore a fun and interesting use-case of recurrent
sequence-to-sequence models. We will train a simple chatbot using movie
scripts from the [Kaggle](https://www.kaggle.com/datasets/isunil/questionsanswers?select=questions.txt)_.

Conversational models are a hot topic in artificial intelligence
research. Chatbots can be found in a variety of settings, including
customer service applications and online helpdesks. These bots are often
powered by retrieval-based models, which output predefined responses to
questions of certain forms. In a highly restricted domain like a
company’s IT helpdesk, these models may be sufficient, however, they are
not robust enough for more general use-cases. Teaching a machine to
carry out a meaningful conversation with a human in multiple domains is
a research question that is far from solved. Recently, the deep learning
boom has allowed for powerful generative models like Google’s [Neural
Conversational Model](https://arxiv.org/abs/1506.05869)_, which marks
a large step towards multi-domain generative conversational models. In
this tutorial, we will implement this kind of model in PyTorch.

.. figure:: /_static/img/chatbot/bot.png
   :align: center
   :alt: bot

```python
> hello?
Bot: hello .
> where am I?
> goodbye
Bot: goodbye .
```
**Acknowledgments**

This tutorial borrows code from the following sources:

1) Yuan-Kuei Wu’s pytorch-chatbot implementation:
   https://github.com/ywk991112/pytorch-chatbot

2) Sean Robertson’s practical-pytorch seq2seq-translation example:
   https://github.com/spro/practical-pytorch/tree/master/seq2seq-translation



## Preparations

To start, Download the data ZIP file
[here](https://www.kaggle.com/code/letanpha/pre-determined-chatbot-ai)_



In [2]:
# and put in a ``data/`` directory under the current directory.
#
# After that, let’s import some necessities.
#

import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
from io import open
import itertools
import math
import time


USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")

## Load & Preprocess Data

The next step is to reformat our data file and load the data into
structures that we can work with.

In [3]:
corpus_name = "kaggle"
corpus = os.path.join("data", corpus_name)

### Merged data file

In [4]:
questions = []
answers = []
print("Reading lines...")
with open(f'{corpus}/questions.txt', encoding='utf-8') as questions_file:
    questions = questions_file.read().strip().split('\n')
with open(f'{corpus}/answers.txt', encoding='utf-8') as answers_file:
    answers = answers_file.read().strip().split('\n')

print("Check length of q and a... :", len(questions) == len(answers))


Now we’ll call these functions and create the file. We’ll call it
``megred_data.txt``.




In [5]:
# # Define path to new file
datafile = os.path.join(corpus, "megred_data.txt")

delimiter = '\t'
# Unescape the delimiter
delimiter = str(codecs.decode(delimiter, "unicode_escape"))

# Write new csv file
print("\nWriting newly formatted file...")
with open(datafile, 'w', encoding='utf-8') as outputfile:
    writer = csv.writer(outputfile, delimiter=delimiter, lineterminator='\n')
    for i in range(len(questions)):
        writer.writerow([questions[i], answers[i]])

# Print a sample of lines
print("\nSample lines from file:")
printLines(datafile)

### Load and trim data

Our next order of business is to create a vocabulary and load
query/response sentence pairs into memory.

Note that we are dealing with sequences of **words**, which do not have
an implicit mapping to a discrete numerical space. Thus, we must create
one by mapping each unique word that we encounter in our dataset to an
index value.

For this we define a ``Voc`` class, which keeps a mapping from words to
indexes, a reverse mapping of indexes to words, a count of each word and
a total word count. The class provides methods for adding a word to the
vocabulary (``addWord``), adding all words in a sentence
(``addSentence``) and trimming infrequently seen words (``trim``). More
on trimming later.




In [6]:
# Default word tokens
PAD_token = 0  # Used for padding short sentences
SOS_token = 1  # Start-of-sentence token
EOS_token = 2  # End-of-sentence token

class Voc:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3  # Count SOS, EOS, PAD

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1

    # Remove words below a certain count threshold
    def trim(self, min_count):
        if self.trimmed:
            return
        self.trimmed = True

        keep_words = []

        for k, v in self.word2count.items():
            if v >= min_count:
                keep_words.append(k)

        print('keep_words {} / {} = {:.4f}'.format(
            len(keep_words), len(self.word2index), len(keep_words) / len(self.word2index)
        ))

        # Reinitialize dictionaries
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3 # Count default tokens

        for word in keep_words:
            self.addWord(word)

Now we can assemble our vocabulary and query/response sentence pairs.
Before we are ready to use this data, we must perform some
preprocessing.

First, we must convert the Unicode strings to ASCII using
``unicodeToAscii``. Next, we should convert all letters to lowercase and
trim all non-letter characters except for basic punctuation
(``normalizeString``). Finally, to aid in training convergence, we will
filter out sentences with length greater than the ``MAX_LENGTH``
threshold (``filterPairs``).




In [7]:
MAX_LENGTH = 20  # Maximum sentence length to consider

# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()
    return s

# Read query/response pairs and return a voc object
def readVocs(datafile, corpus_name):
    print("Reading lines...")
    # Read the file and split into lines
    lines = open(datafile, encoding='utf-8').\
        read().strip().split('\n')
    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
    voc = Voc(corpus_name)
    return voc, pairs

# Returns True if both sentences in a pair 'p' are under the MAX_LENGTH threshold
def filterPair(p):
    # Input sequences need to preserve the last word for EOS token
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH

# Filter pairs using the ``filterPair`` condition
def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

# Using the functions defined above, return a populated voc object and pairs list
def loadPrepareData(corpus, corpus_name, datafile, save_dir):
    print("Start preparing training data ...")
    voc, pairs = readVocs(datafile, corpus_name)
    print("Read {!s} sentence pairs".format(len(pairs)))
    pairs = filterPairs(pairs)
    print("Trimmed to {!s} sentence pairs".format(len(pairs)))
    print("Counting words...")
    for pair in pairs:
        voc.addSentence(pair[0])
        voc.addSentence(pair[1])
    print("Counted words:", voc.num_words)
    return voc, pairs

# Load/Assemble voc and pairs
save_dir = os.path.join("data", "save")
voc, pairs = loadPrepareData(corpus, corpus_name, datafile, save_dir)
# Print some pairs to validate
print("\npairs:")
for pair in pairs[:10]:
    print(pair)

Start preparing training data ...
Reading lines...
Read 221616 sentence pairs
Trimmed to 162446 sentence pairs
Counting words...
Counted words: 35099

pairs:
['well i thought we would start with pronunciation if that is okay with you', 'not the hacking and gagging and spitting part please']
['not the hacking and gagging and spitting part please', 'okay then how bout we try out some french cuisine saturday night']
['you are asking me out that is so cute what is your name again', 'forget it']
['no no it s my fault we didnot have a proper introduction', 'cameron']
['gosh if only we could find kat a boyfriend', 'let me see what i can do']
['c esc ma tete this is my head', 'right see you are ready for the quiz']
['that is because it s such a nice one', 'forget french']
['how is our little find the wench a date plan progressing', 'well there s someone i think might be']
['there', 'where']
['you have my word as a gentleman', 'you are sweet']


In [27]:
#save tokenizer
import pickle

with open('data/save/cb_model/voc_tokenizer.pkl', 'wb') as out:
    pickle.dump(voc, out, pickle.HIGHEST_PROTOCOL)

Another tactic that is beneficial to achieving faster convergence during
training is trimming rarely used words out of our vocabulary. Decreasing
the feature space will also soften the difficulty of the function that
the model must learn to approximate. We will do this as a two-step
process:

1) Trim words used under ``MIN_COUNT`` threshold using the ``voc.trim``
   function.

2) Filter out pairs with trimmed words.




In [8]:
MIN_COUNT = 3    # Minimum word count threshold for trimming

def trimRareWords(voc, pairs, MIN_COUNT):
    # Trim words used under the MIN_COUNT from the voc
    voc.trim(MIN_COUNT)
    # Filter out pairs with trimmed words
    keep_pairs = []
    for pair in pairs:
        input_sentence = pair[0]
        output_sentence = pair[1]
        keep_input = True
        keep_output = True
        # Check input sentence
        for word in input_sentence.split(' '):
            if word not in voc.word2index:
                keep_input = False
                break
        # Check output sentence
        for word in output_sentence.split(' '):
            if word not in voc.word2index:
                keep_output = False
                break

        # Only keep pairs that do not contain trimmed word(s) in their input or output sentence
        if keep_input and keep_output:
            keep_pairs.append(pair)

    print("Trimmed from {} pairs to {}, {:.4f} of total".format(len(pairs), len(keep_pairs), len(keep_pairs) / len(pairs)))
    return keep_pairs


# Trim voc and pairs
pairs = trimRareWords(voc, pairs, MIN_COUNT)

keep_words 18706 / 35096 = 0.5330
Trimmed from 162446 pairs to 143317, 0.8822 of total


## Prepare Data for Models

Although we have put a great deal of effort into preparing and massaging our
data into a nice vocabulary object and list of sentence pairs, our models
will ultimately expect numerical torch tensors as inputs. One way to
prepare the processed data for the models can be found in the [seq2seq
translation
tutorial](https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html)_.
In that tutorial, we use a batch size of 1, meaning that all we have to
do is convert the words in our sentence pairs to their corresponding
indexes from the vocabulary and feed this to the models.

However, if you’re interested in speeding up training and/or would like
to leverage GPU parallelization capabilities, you will need to train
with mini-batches.

Using mini-batches also means that we must be mindful of the variation
of sentence length in our batches. To accommodate sentences of different
sizes in the same batch, we will make our batched input tensor of shape
*(max_length, batch_size)*, where sentences shorter than the
*max_length* are zero padded after an *EOS_token*.

If we simply convert our English sentences to tensors by converting
words to their indexes(\ ``indexesFromSentence``) and zero-pad, our
tensor would have shape *(batch_size, max_length)* and indexing the
first dimension would return a full sequence across all time-steps.
However, we need to be able to index our batch along time, and across
all sequences in the batch. Therefore, we transpose our input batch
shape to *(max_length, batch_size)*, so that indexing across the first
dimension returns a time step across all sentences in the batch. We
handle this transpose implicitly in the ``zeroPadding`` function.

.. figure:: /_static/img/chatbot/seq2seq_batches.png
   :align: center
   :alt: batches

The ``inputVar`` function handles the process of converting sentences to
tensor, ultimately creating a correctly shaped zero-padded tensor. It
also returns a tensor of ``lengths`` for each of the sequences in the
batch which will be passed to our decoder later.

The ``outputVar`` function performs a similar function to ``inputVar``,
but instead of returning a ``lengths`` tensor, it returns a binary mask
tensor and a maximum target sentence length. The binary mask tensor has
the same shape as the output target tensor, but every element that is a
*PAD_token* is 0 and all others are 1.

``batch2TrainData`` simply takes a bunch of pairs and returns the input
and target tensors using the aforementioned functions.




In [9]:
def indexesFromSentence(voc, sentence):
    return [voc.word2index[word] for word in sentence.split(' ')] + [EOS_token]


def zeroPadding(l, fillvalue=PAD_token):
    return list(itertools.zip_longest(*l, fillvalue=fillvalue))

def binaryMatrix(l, value=PAD_token):
    m = []
    for i, seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == PAD_token:
                m[i].append(0)
            else:
                m[i].append(1)
    return m

# Returns padded input sequence tensor and lengths
def inputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    padVar = torch.LongTensor(padList)
    return padVar, lengths

# Returns padded target sequence tensor, padding mask, and max target length
def outputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    max_target_len = max([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    mask = binaryMatrix(padList)
    mask = torch.BoolTensor(mask)
    padVar = torch.LongTensor(padList)
    return padVar, mask, max_target_len

# Returns all items for a given batch of pairs
def batch2TrainData(voc, pair_batch):
    pair_batch.sort(key=lambda x: len(x[0].split(" ")), reverse=True)
    input_batch, output_batch = [], []
    for pair in pair_batch:
        input_batch.append(pair[0])
        output_batch.append(pair[1])
    inp, lengths = inputVar(input_batch, voc)
    output, mask, max_target_len = outputVar(output_batch, voc)
    return inp, lengths, output, mask, max_target_len


# Example for validation
small_batch_size = 5
batches = batch2TrainData(voc, [random.choice(pairs) for _ in range(small_batch_size)])
input_variable, lengths, target_variable, mask, max_target_len = batches

print("input_variable:", input_variable)
print("lengths:", lengths)
print("target_variable:", target_variable)
print("mask:", mask)
print("max_target_len:", max_target_len)

input_variable: tensor([[    4,  1959,    12,    23,    38],
        [   83,   207,    11,   298,     2],
        [   14,   458,   516,     2,     0],
        [  128,    16,  2160,     0,     0],
        [   38, 12471,     2,     0,     0],
        [    4,    91,     0,     0,     0],
        [  257,  6207,     0,     0,     0],
        [    2,     2,     0,     0,     0]])
lengths: tensor([8, 8, 5, 3, 2])
target_variable: tensor([[  44,   24,   44,   72,    4],
        [   4, 2723,    2,  193,  106],
        [ 106,   45,    0,   12,  449],
        [ 184,  507,    0,   15, 6133],
        [   2, 2246,    0,  731,    2],
        [   0,  449,    0,  177,    0],
        [   0,   85,    0,   35,    0],
        [   0,    2,    0,    2,    0]])
mask: tensor([[ True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True],
        [ True,  True, False,  True,  True],
        [ True,  True, False,  True,  True],
        [ True,  True, False,  True,  True],
        [False,  True

## Define Models

### Seq2Seq Model

The brains of our chatbot is a sequence-to-sequence (seq2seq) model. The
goal of a seq2seq model is to take a variable-length sequence as an
input, and return a variable-length sequence as an output using a
fixed-sized model.

[Sutskever et al.](https://arxiv.org/abs/1409.3215)_ discovered that
by using two separate recurrent neural nets together, we can accomplish
this task. One RNN acts as an **encoder**, which encodes a variable
length input sequence to a fixed-length context vector. In theory, this
context vector (the final hidden layer of the RNN) will contain semantic
information about the query sentence that is input to the bot. The
second RNN is a **decoder**, which takes an input word and the context
vector, and returns a guess for the next word in the sequence and a
hidden state to use in the next iteration.

.. figure:: /_static/img/chatbot/seq2seq_ts.png
   :align: center
   :alt: model

Image source:
https://jeddy92.github.io/JEddy92.github.io/ts_seq2seq_intro/




### Encoder

The encoder RNN iterates through the input sentence one token
(e.g. word) at a time, at each time step outputting an “output” vector
and a “hidden state” vector. The hidden state vector is then passed to
the next time step, while the output vector is recorded. The encoder
transforms the context it saw at each point in the sequence into a set
of points in a high-dimensional space, which the decoder will use to
generate a meaningful output for the given task.

At the heart of our encoder is a multi-layered Gated Recurrent Unit,
invented by [Cho et al.](https://arxiv.org/pdf/1406.1078v3.pdf)_ in
2014. We will use a bidirectional variant of the GRU, meaning that there
are essentially two independent RNNs: one that is fed the input sequence
in normal sequential order, and one that is fed the input sequence in
reverse order. The outputs of each network are summed at each time step.
Using a bidirectional GRU will give us the advantage of encoding both
past and future contexts.

Bidirectional RNN:

.. figure:: /_static/img/chatbot/RNN-bidirectional.png
   :width: 70%
   :align: center
   :alt: rnn_bidir

Image source: https://colah.github.io/posts/2015-09-NN-Types-FP/

Note that an ``embedding`` layer is used to encode our word indices in
an arbitrarily sized feature space. For our models, this layer will map
each word to a feature space of size *hidden_size*. When trained, these
values should encode semantic similarity between similar meaning words.

Finally, if passing a padded batch of sequences to an RNN module, we
must pack and unpack padding around the RNN pass using
``nn.utils.rnn.pack_padded_sequence`` and
``nn.utils.rnn.pad_packed_sequence`` respectively.

**Computation Graph:**

   1) Convert word indexes to embeddings.
   2) Pack padded batch of sequences for RNN module.
   3) Forward pass through GRU.
   4) Unpack padding.
   5) Sum bidirectional GRU outputs.
   6) Return output and final hidden state.

**Inputs:**

-  ``input_seq``: batch of input sentences; shape=\ *(max_length,
   batch_size)*
-  ``input_lengths``: list of sentence lengths corresponding to each
   sentence in the batch; shape=\ *(batch_size)*
-  ``hidden``: hidden state; shape=\ *(n_layers x num_directions,
   batch_size, hidden_size)*

**Outputs:**

-  ``outputs``: output features from the last hidden layer of the GRU
   (sum of bidirectional outputs); shape=\ *(max_length, batch_size,
   hidden_size)*
-  ``hidden``: updated hidden state from GRU; shape=\ *(n_layers x
   num_directions, batch_size, hidden_size)*





In [10]:
class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, embedding, n_layers=1, dropout=0):
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = embedding

        # Initialize GRU; the input_size and hidden_size parameters are both set to 'hidden_size'
        #   because our input size is a word embedding with number of features == hidden_size
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers,
                          dropout=(0 if n_layers == 1 else dropout), bidirectional=True)

    def forward(self, input_seq, input_lengths, hidden=None):
        # Convert word indexes to embeddings
        embedded = self.embedding(input_seq)
        # Pack padded batch of sequences for RNN module
        packed = nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
        # Forward pass through GRU
        outputs, hidden = self.gru(packed, hidden)
        # Unpack padding
        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs)
        # Sum bidirectional GRU outputs
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:]
        # Return output and final hidden state
        return outputs, hidden

### Decoder

The decoder RNN generates the response sentence in a token-by-token
fashion. It uses the encoder’s context vectors, and internal hidden
states to generate the next word in the sequence. It continues
generating words until it outputs an *EOS_token*, representing the end
of the sentence. A common problem with a vanilla seq2seq decoder is that
if we rely solely on the context vector to encode the entire input
sequence’s meaning, it is likely that we will have information loss.
This is especially the case when dealing with long input sequences,
greatly limiting the capability of our decoder.

To combat this, [Bahdanau et al.](https://arxiv.org/abs/1409.0473)_
created an “attention mechanism” that allows the decoder to pay
attention to certain parts of the input sequence, rather than using the
entire fixed context at every step.

At a high level, attention is calculated using the decoder’s current
hidden state and the encoder’s outputs. The output attention weights
have the same shape as the input sequence, allowing us to multiply them
by the encoder outputs, giving us a weighted sum which indicates the
parts of encoder output to pay attention to. [Sean
Robertson’s](https://github.com/spro)_ figure describes this very
well:

.. figure:: /_static/img/chatbot/attn2.png
   :align: center
   :alt: attn2

[Luong et al.](https://arxiv.org/abs/1508.04025)_ improved upon
Bahdanau et al.’s groundwork by creating “Global attention”. The key
difference is that with “Global attention”, we consider all of the
encoder’s hidden states, as opposed to Bahdanau et al.’s “Local
attention”, which only considers the encoder’s hidden state from the
current time step. Another difference is that with “Global attention”,
we calculate attention weights, or energies, using the hidden state of
the decoder from the current time step only. Bahdanau et al.’s attention
calculation requires knowledge of the decoder’s state from the previous
time step. Also, Luong et al. provides various methods to calculate the
attention energies between the encoder output and decoder output which
are called “score functions”:

.. figure:: /_static/img/chatbot/scores.png
   :width: 60%
   :align: center
   :alt: scores

where $h_t$ = current target decoder state and $\bar{h}_s$ =
all encoder states.

Overall, the Global attention mechanism can be summarized by the
following figure. Note that we will implement the “Attention Layer” as a
separate ``nn.Module`` called ``Attn``. The output of this module is a
softmax normalized weights tensor of shape *(batch_size, 1,
max_length)*.

.. figure:: /_static/img/chatbot/global_attn.png
   :align: center
   :width: 60%
   :alt: global_attn




In [11]:
# Luong attention layer
class Attn(nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method
        if self.method not in ['dot', 'general', 'concat']:
            raise ValueError(self.method, "is not an appropriate attention method.")
        self.hidden_size = hidden_size
        if self.method == 'general':
            self.attn = nn.Linear(self.hidden_size, hidden_size)
        elif self.method == 'concat':
            self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
            self.v = nn.Parameter(torch.FloatTensor(hidden_size))

    def dot_score(self, hidden, encoder_output):
        return torch.sum(hidden * encoder_output, dim=2)

    def general_score(self, hidden, encoder_output):
        energy = self.attn(encoder_output)
        return torch.sum(hidden * energy, dim=2)

    def concat_score(self, hidden, encoder_output):
        energy = self.attn(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), encoder_output), 2)).tanh()
        return torch.sum(self.v * energy, dim=2)

    def forward(self, hidden, encoder_outputs):
        # Calculate the attention weights (energies) based on the given method
        if self.method == 'general':
            attn_energies = self.general_score(hidden, encoder_outputs)
        elif self.method == 'concat':
            attn_energies = self.concat_score(hidden, encoder_outputs)
        elif self.method == 'dot':
            attn_energies = self.dot_score(hidden, encoder_outputs)

        # Transpose max_length and batch_size dimensions
        attn_energies = attn_energies.t()

        # Return the softmax normalized probability scores (with added dimension)
        return F.softmax(attn_energies, dim=1).unsqueeze(1)

Now that we have defined our attention submodule, we can implement the
actual decoder model. For the decoder, we will manually feed our batch
one time step at a time. This means that our embedded word tensor and
GRU output will both have shape *(1, batch_size, hidden_size)*.

**Computation Graph:**

   1) Get embedding of current input word.
   2) Forward through unidirectional GRU.
   3) Calculate attention weights from the current GRU output from (2).
   4) Multiply attention weights to encoder outputs to get new "weighted sum" context vector.
   5) Concatenate weighted context vector and GRU output using Luong eq. 5.
   6) Predict next word using Luong eq. 6 (without softmax).
   7) Return output and final hidden state.

**Inputs:**

-  ``input_step``: one time step (one word) of input sequence batch;
   shape=\ *(1, batch_size)*
-  ``last_hidden``: final hidden layer of GRU; shape=\ *(n_layers x
   num_directions, batch_size, hidden_size)*
-  ``encoder_outputs``: encoder model’s output; shape=\ *(max_length,
   batch_size, hidden_size)*

**Outputs:**

-  ``output``: softmax normalized tensor giving probabilities of each
   word being the correct next word in the decoded sequence;
   shape=\ *(batch_size, voc.num_words)*
-  ``hidden``: final hidden state of GRU; shape=\ *(n_layers x
   num_directions, batch_size, hidden_size)*




In [12]:
class LuongAttnDecoderRNN(nn.Module):
    def __init__(self, attn_model, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):
        super(LuongAttnDecoderRNN, self).__init__()

        # Keep for reference
        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout

        # Define layers
        self.embedding = embedding
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout))
        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)

        self.attn = Attn(attn_model, hidden_size)

    def forward(self, input_step, last_hidden, encoder_outputs):
        # Note: we run this one step (word) at a time
        # Get embedding of current input word
        embedded = self.embedding(input_step)
        embedded = self.embedding_dropout(embedded)
        # Forward through unidirectional GRU
        rnn_output, hidden = self.gru(embedded, last_hidden)
        # Calculate attention weights from the current GRU output
        attn_weights = self.attn(rnn_output, encoder_outputs)
        # Multiply attention weights to encoder outputs to get new "weighted sum" context vector
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1))
        # Concatenate weighted context vector and GRU output using Luong eq. 5
        rnn_output = rnn_output.squeeze(0)
        context = context.squeeze(1)
        concat_input = torch.cat((rnn_output, context), 1)
        concat_output = torch.tanh(self.concat(concat_input))
        # Predict next word using Luong eq. 6
        output = self.out(concat_output)
        output = F.softmax(output, dim=1)
        # Return output and final hidden state
        return output, hidden

## Define Training Procedure

### Masked loss

Since we are dealing with batches of padded sequences, we cannot simply
consider all elements of the tensor when calculating loss. We define
``maskNLLLoss`` to calculate our loss based on our decoder’s output
tensor, the target tensor, and a binary mask tensor describing the
padding of the target tensor. This loss function calculates the average
negative log likelihood of the elements that correspond to a *1* in the
mask tensor.




In [13]:
def maskNLLLoss(inp, target, mask):
    nTotal = mask.sum()
    crossEntropy = -torch.log(torch.gather(inp, 1, target.view(-1, 1)).squeeze(1))
    loss = crossEntropy.masked_select(mask).mean()
    loss = loss.to(device)
    return loss, nTotal.item()

### Single training iteration

The ``train`` function contains the algorithm for a single training
iteration (a single batch of inputs).

We will use a couple of clever tricks to aid in convergence:

-  The first trick is using **teacher forcing**. This means that at some
   probability, set by ``teacher_forcing_ratio``, we use the current
   target word as the decoder’s next input rather than using the
   decoder’s current guess. This technique acts as training wheels for
   the decoder, aiding in more efficient training. However, teacher
   forcing can lead to model instability during inference, as the
   decoder may not have a sufficient chance to truly craft its own
   output sequences during training. Thus, we must be mindful of how we
   are setting the ``teacher_forcing_ratio``, and not be fooled by fast
   convergence.

-  The second trick that we implement is **gradient clipping**. This is
   a commonly used technique for countering the “exploding gradient”
   problem. In essence, by clipping or thresholding gradients to a
   maximum value, we prevent the gradients from growing exponentially
   and either overflow (NaN), or overshoot steep cliffs in the cost
   function.

.. figure:: /_static/img/chatbot/grad_clip.png
   :align: center
   :width: 60%
   :alt: grad_clip

Image source: Goodfellow et al. *Deep Learning*. 2016. https://www.deeplearningbook.org/

**Sequence of Operations:**

   1) Forward pass entire input batch through encoder.
   2) Initialize decoder inputs as SOS_token, and hidden state as the encoder's final hidden state.
   3) Forward input batch sequence through decoder one time step at a time.
   4) If teacher forcing: set next decoder input as the current target; else: set next decoder input as current decoder output.
   5) Calculate and accumulate loss.
   6) Perform backpropagation.
   7) Clip gradients.
   8) Update encoder and decoder model parameters.


.. Note ::

  PyTorch’s RNN modules (``RNN``, ``LSTM``, ``GRU``) can be used like any
  other non-recurrent layers by simply passing them the entire input
  sequence (or batch of sequences). We use the ``GRU`` layer like this in
  the ``encoder``. The reality is that under the hood, there is an
  iterative process looping over each time step calculating hidden states.
  Alternatively, you can run these modules one time-step at a time. In
  this case, we manually loop over the sequences during the training
  process like we must do for the ``decoder`` model. As long as you
  maintain the correct conceptual model of these modules, implementing
  sequential models can be very straightforward.





In [14]:
def train(input_variable, lengths, target_variable, mask, max_target_len, encoder, decoder, embedding,
          encoder_optimizer, decoder_optimizer, batch_size, clip, max_length=MAX_LENGTH):

    # Zero gradients
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    # Set device options
    input_variable = input_variable.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)
    # Lengths for RNN packing should always be on the CPU
    lengths = lengths.to("cpu")

    # Initialize variables
    loss = 0
    print_losses = []
    n_totals = 0

    # Forward pass through encoder
    encoder_outputs, encoder_hidden = encoder(input_variable, lengths)

    # Create initial decoder input (start with SOS tokens for each sentence)
    decoder_input = torch.LongTensor([[SOS_token for _ in range(batch_size)]])
    decoder_input = decoder_input.to(device)

    # Set initial decoder hidden state to the encoder's final hidden state
    decoder_hidden = encoder_hidden[:decoder.n_layers]

    # Determine if we are using teacher forcing this iteration
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    # Forward batch of sequences through decoder one time step at a time
    if use_teacher_forcing:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # Teacher forcing: next input is current target
            decoder_input = target_variable[t].view(1, -1)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal
    else:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # No teacher forcing: next input is decoder's own current output
            _, topi = decoder_output.topk(1)
            decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
            decoder_input = decoder_input.to(device)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal

    # Perform backpropagation
    loss.backward()

    # Clip gradients: gradients are modified in place
    _ = nn.utils.clip_grad_norm_(encoder.parameters(), clip)
    _ = nn.utils.clip_grad_norm_(decoder.parameters(), clip)

    # Adjust model weights
    encoder_optimizer.step()
    decoder_optimizer.step()

    return sum(print_losses) / n_totals

### Training iterations

It is finally time to tie the full training procedure together with the
data. The ``trainIters`` function is responsible for running
``n_iterations`` of training given the passed models, optimizers, data,
etc. This function is quite self explanatory, as we have done the heavy
lifting with the ``train`` function.

One thing to note is that when we save our model, we save a tarball
containing the encoder and decoder ``state_dicts`` (parameters), the
optimizers’ ``state_dicts``, the loss, the iteration, etc. Saving the model
in this way will give us the ultimate flexibility with the checkpoint.
After loading a checkpoint, we will be able to use the model parameters
to run inference, or we can continue training right where we left off.




In [15]:
def trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer, embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size, print_every, save_every, clip, corpus_name, loadFilename):

    # Load batches for each iteration
    training_batches = [batch2TrainData(voc, [random.choice(pairs) for _ in range(batch_size)])
                      for _ in range(n_iteration)]

    # Initializations
    print('Initializing ...')
    start_iteration = 1
    print_loss = 0
    if loadFilename:
        start_iteration = checkpoint['iteration'] + 1

    # Training loop
    print("Training...")
    for iteration in range(start_iteration, n_iteration + 1):
        training_batch = training_batches[iteration - 1]
        # Extract fields from batch
        input_variable, lengths, target_variable, mask, max_target_len = training_batch

        # Run a training iteration with batch
        loss = train(input_variable, lengths, target_variable, mask, max_target_len, encoder,
                     decoder, embedding, encoder_optimizer, decoder_optimizer, batch_size, clip)
        print_loss += loss

        # Print progress
        if iteration % print_every == 0:
            print_loss_avg = print_loss / print_every
            print("Iteration: {}; Percent complete: {:.1f}%; Average loss: {:.4f}".format(iteration, iteration / n_iteration * 100, print_loss_avg))
            print_loss = 0

        # Save checkpoint
        if (iteration % save_every == 0):
            directory = os.path.join(save_dir, model_name, corpus_name, '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size))
            if not os.path.exists(directory):
                os.makedirs(directory)
            torch.save({
                'iteration': iteration,
                'en': encoder.state_dict(),
                'de': decoder.state_dict(),
                'en_opt': encoder_optimizer.state_dict(),
                'de_opt': decoder_optimizer.state_dict(),
                'loss': loss,
                'voc_dict': voc.__dict__,
                'embedding': embedding.state_dict()
            }, os.path.join(directory, '{}_{}.tar'.format(iteration, 'checkpoint')))

## Define Evaluation

After training a model, we want to be able to talk to the bot ourselves.
First, we must define how we want the model to decode the encoded input.

### Greedy decoding

Greedy decoding is the decoding method that we use during training when
we are **NOT** using teacher forcing. In other words, for each time
step, we simply choose the word from ``decoder_output`` with the highest
softmax value. This decoding method is optimal on a single time-step
level.

To facilitate the greedy decoding operation, we define a
``GreedySearchDecoder`` class. When run, an object of this class takes
an input sequence (``input_seq``) of shape *(input_seq length, 1)*, a
scalar input length (``input_length``) tensor, and a ``max_length`` to
bound the response sentence length. The input sentence is evaluated
using the following computational graph:

**Computation Graph:**

   1) Forward input through encoder model.
   2) Prepare encoder's final hidden layer to be first hidden input to the decoder.
   3) Initialize decoder's first input as SOS_token.
   4) Initialize tensors to append decoded words to.
   5) Iteratively decode one word token at a time:
       a) Forward pass through decoder.
       b) Obtain most likely word token and its softmax score.
       c) Record token and score.
       d) Prepare current token to be next decoder input.
   6) Return collections of word tokens and scores.




In [16]:
class GreedySearchDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super(GreedySearchDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, input_seq, input_length, max_length):
        # Forward input through encoder model
        encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length)
        # Prepare encoder's final hidden layer to be first hidden input to the decoder
        decoder_hidden = encoder_hidden[:decoder.n_layers]
        # Initialize decoder input with SOS_token
        decoder_input = torch.ones(1, 1, device=device, dtype=torch.long) * SOS_token
        # Initialize tensors to append decoded words to
        all_tokens = torch.zeros([0], device=device, dtype=torch.long)
        all_scores = torch.zeros([0], device=device)
        # Iteratively decode one word token at a time
        for _ in range(max_length):
            # Forward pass through decoder
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
            # Obtain most likely word token and its softmax score
            decoder_scores, decoder_input = torch.max(decoder_output, dim=1)
            # Record token and score
            all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
            all_scores = torch.cat((all_scores, decoder_scores), dim=0)
            # Prepare current token to be next decoder input (add a dimension)
            decoder_input = torch.unsqueeze(decoder_input, 0)
        # Return collections of word tokens and scores
        return all_tokens, all_scores

### Evaluate my text

Now that we have our decoding method defined, we can write functions for
evaluating a string input sentence. The ``evaluate`` function manages
the low-level process of handling the input sentence. We first format
the sentence as an input batch of word indexes with *batch_size==1*. We
do this by converting the words of the sentence to their corresponding
indexes, and transposing the dimensions to prepare the tensor for our
models. We also create a ``lengths`` tensor which contains the length of
our input sentence. In this case, ``lengths`` is scalar because we are
only evaluating one sentence at a time (batch_size==1). Next, we obtain
the decoded response sentence tensor using our ``GreedySearchDecoder``
object (``searcher``). Finally, we convert the response’s indexes to
words and return the list of decoded words.

``evaluateInput`` acts as the user interface for our chatbot. When
called, an input text field will spawn in which we can enter our query
sentence. After typing our input sentence and pressing *Enter*, our text
is normalized in the same way as our training data, and is ultimately
fed to the ``evaluate`` function to obtain a decoded output sentence. We
loop this process, so we can keep chatting with our bot until we enter
either “q” or “quit”.

Finally, if a sentence is entered that contains a word that is not in
the vocabulary, we handle this gracefully by printing an error message
and prompting the user to enter another sentence.




In [17]:
def evaluate(encoder, decoder, searcher, voc, sentence, max_length=MAX_LENGTH):
    ### Format input sentence as a batch
    # words -> indexes
    indexes_batch = [indexesFromSentence(voc, sentence)]
    # Create lengths tensor
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    # Transpose dimensions of batch to match models' expectations
    input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
    # Use appropriate device
    input_batch = input_batch.to(device)
    lengths = lengths.to("cpu")
    # Decode sentence with searcher
    tokens, scores = searcher(input_batch, lengths, max_length)
    # indexes -> words
    decoded_words = [voc.index2word[token.item()] for token in tokens]
    return decoded_words


def evaluateInput(encoder, decoder, searcher, voc):
    input_sentence = ''
    while(1):
        try:
            # Get input sentence
            input_sentence = input('> ')
            # Check if it is quit case
            if input_sentence == 'q' or input_sentence == 'quit': break
            # Normalize sentence
            input_sentence = normalizeString(input_sentence)
            # Evaluate sentence
            output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
            # Format and print response sentence
            output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
            print('Bot:', ' '.join(output_words))

        except KeyError:
            print("Error: Encountered unknown word.")

## Run Model

Finally, it is time to run our model!

Regardless of whether we want to train or test the chatbot model, we
must initialize the individual encoder and decoder models. In the
following block, we set our desired configurations, choose to start from
scratch or set a checkpoint to load from, and build and initialize the
models. Feel free to play with different model configurations to
optimize performance.




In [18]:
# Configure models
model_name = 'cb_model'
attn_model = 'dot'
#``attn_model = 'general'``
#``attn_model = 'concat'``
hidden_size = 500
encoder_n_layers = 2
decoder_n_layers = 2
dropout = 0.1
batch_size = 256

# Set checkpoint to load from; set to None if starting from scratch
loadFilename = None
checkpoint_iter = 4000

Sample code to load from a checkpoint:

```python
loadFilename = os.path.join(save_dir, model_name, corpus_name,
                    '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size),
                    '{}_checkpoint.tar'.format(checkpoint_iter))
```


In [19]:
# Load model if a ``loadFilename`` is provided
if loadFilename:
    # If loading on same machine the model was trained on
    checkpoint = torch.load(loadFilename)
    # If loading a model trained on GPU to CPU
    #checkpoint = torch.load(loadFilename, map_location=torch.device('cpu'))
    encoder_sd = checkpoint['en']
    decoder_sd = checkpoint['de']
    encoder_optimizer_sd = checkpoint['en_opt']
    decoder_optimizer_sd = checkpoint['de_opt']
    embedding_sd = checkpoint['embedding']
    voc.__dict__ = checkpoint['voc_dict']


print('Building encoder and decoder ...')
# Initialize word embeddings
embedding = nn.Embedding(voc.num_words, hidden_size)
if loadFilename:
    embedding.load_state_dict(embedding_sd)
# Initialize encoder & decoder models
encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, voc.num_words, decoder_n_layers, dropout)
if loadFilename:
    encoder.load_state_dict(encoder_sd)
    decoder.load_state_dict(decoder_sd)
# Use appropriate device
encoder = encoder.to(device)
decoder = decoder.to(device)
print('Models built and ready to go!')

Building encoder and decoder ...
Models built and ready to go!


In [25]:
print(hidden_size)
print(embedding) 
print(encoder_n_layers)
print(dropout)

500
Embedding(18709, 500)
2
0.1


### Run Training

Run the following block if you want to train the model.

First we set training parameters, then we initialize our optimizers, and
finally we call the ``trainIters`` function to run our training
iterations.




In [20]:
# Configure training/optimization
clip = 50.0
teacher_forcing_ratio = 1.0
learning_rate = 0.0001
decoder_learning_ratio = 5.0
n_iteration = 4000
print_every = 1
save_every = 500

# Ensure dropout layers are in train mode
encoder.train()
decoder.train()

# Initialize optimizers
print('Building optimizers ...')
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)
if loadFilename:
    encoder_optimizer.load_state_dict(encoder_optimizer_sd)
    decoder_optimizer.load_state_dict(decoder_optimizer_sd)

# If you have CUDA, configure CUDA to call
for state in encoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()

for state in decoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()

# Run training iterations
print("Starting Training!")
s_time = time.time()
trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer,
           embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size,
           print_every, save_every, clip, corpus_name, loadFilename)
print("done. Total time: ", time.time() - s_time)

Building optimizers ...
Starting Training!
Initializing ...
Training...
Iteration: 1; Percent complete: 0.0%; Average loss: 9.8309
Iteration: 2; Percent complete: 0.1%; Average loss: 9.7868
Iteration: 3; Percent complete: 0.1%; Average loss: 9.7273
Iteration: 4; Percent complete: 0.1%; Average loss: 9.5918
Iteration: 5; Percent complete: 0.1%; Average loss: 9.3735
Iteration: 6; Percent complete: 0.1%; Average loss: 8.9915
Iteration: 7; Percent complete: 0.2%; Average loss: 8.5047
Iteration: 8; Percent complete: 0.2%; Average loss: 8.1503
Iteration: 9; Percent complete: 0.2%; Average loss: 8.2157
Iteration: 10; Percent complete: 0.2%; Average loss: 8.1790
Iteration: 11; Percent complete: 0.3%; Average loss: 8.0863
Iteration: 12; Percent complete: 0.3%; Average loss: 7.8319
Iteration: 13; Percent complete: 0.3%; Average loss: 7.6294
Iteration: 14; Percent complete: 0.4%; Average loss: 7.3042
Iteration: 15; Percent complete: 0.4%; Average loss: 7.0728
Iteration: 16; Percent complete: 0.4%

Iteration: 136; Percent complete: 3.4%; Average loss: 5.6619
Iteration: 137; Percent complete: 3.4%; Average loss: 5.6473
Iteration: 138; Percent complete: 3.5%; Average loss: 5.6908
Iteration: 139; Percent complete: 3.5%; Average loss: 5.4923
Iteration: 140; Percent complete: 3.5%; Average loss: 5.6728
Iteration: 141; Percent complete: 3.5%; Average loss: 5.6361
Iteration: 142; Percent complete: 3.5%; Average loss: 5.6942
Iteration: 143; Percent complete: 3.6%; Average loss: 5.5815
Iteration: 144; Percent complete: 3.6%; Average loss: 5.6136
Iteration: 145; Percent complete: 3.6%; Average loss: 5.5834
Iteration: 146; Percent complete: 3.6%; Average loss: 5.5979
Iteration: 147; Percent complete: 3.7%; Average loss: 5.6626
Iteration: 148; Percent complete: 3.7%; Average loss: 5.6663
Iteration: 149; Percent complete: 3.7%; Average loss: 5.5930
Iteration: 150; Percent complete: 3.8%; Average loss: 5.4992
Iteration: 151; Percent complete: 3.8%; Average loss: 5.6706
Iteration: 152; Percent 

Iteration: 271; Percent complete: 6.8%; Average loss: 5.2473
Iteration: 272; Percent complete: 6.8%; Average loss: 5.2424
Iteration: 273; Percent complete: 6.8%; Average loss: 5.2900
Iteration: 274; Percent complete: 6.9%; Average loss: 5.1227
Iteration: 275; Percent complete: 6.9%; Average loss: 5.2172
Iteration: 276; Percent complete: 6.9%; Average loss: 5.1796
Iteration: 277; Percent complete: 6.9%; Average loss: 5.1856
Iteration: 278; Percent complete: 7.0%; Average loss: 5.2155
Iteration: 279; Percent complete: 7.0%; Average loss: 5.3175
Iteration: 280; Percent complete: 7.0%; Average loss: 5.2287
Iteration: 281; Percent complete: 7.0%; Average loss: 5.1780
Iteration: 282; Percent complete: 7.0%; Average loss: 5.1594
Iteration: 283; Percent complete: 7.1%; Average loss: 5.3077
Iteration: 284; Percent complete: 7.1%; Average loss: 5.2192
Iteration: 285; Percent complete: 7.1%; Average loss: 5.1650
Iteration: 286; Percent complete: 7.1%; Average loss: 5.2799
Iteration: 287; Percent 

Iteration: 408; Percent complete: 10.2%; Average loss: 4.9978
Iteration: 409; Percent complete: 10.2%; Average loss: 5.0117
Iteration: 410; Percent complete: 10.2%; Average loss: 4.8828
Iteration: 411; Percent complete: 10.3%; Average loss: 5.0324
Iteration: 412; Percent complete: 10.3%; Average loss: 5.1309
Iteration: 413; Percent complete: 10.3%; Average loss: 5.0562
Iteration: 414; Percent complete: 10.3%; Average loss: 4.9980
Iteration: 415; Percent complete: 10.4%; Average loss: 4.9299
Iteration: 416; Percent complete: 10.4%; Average loss: 5.0959
Iteration: 417; Percent complete: 10.4%; Average loss: 4.8875
Iteration: 418; Percent complete: 10.4%; Average loss: 5.0833
Iteration: 419; Percent complete: 10.5%; Average loss: 5.0712
Iteration: 420; Percent complete: 10.5%; Average loss: 5.0371
Iteration: 421; Percent complete: 10.5%; Average loss: 4.9751
Iteration: 422; Percent complete: 10.5%; Average loss: 4.8491
Iteration: 423; Percent complete: 10.6%; Average loss: 4.9894
Iteratio

Iteration: 541; Percent complete: 13.5%; Average loss: 4.8711
Iteration: 542; Percent complete: 13.6%; Average loss: 4.9567
Iteration: 543; Percent complete: 13.6%; Average loss: 4.9207
Iteration: 544; Percent complete: 13.6%; Average loss: 4.8950
Iteration: 545; Percent complete: 13.6%; Average loss: 4.9260
Iteration: 546; Percent complete: 13.7%; Average loss: 4.9191
Iteration: 547; Percent complete: 13.7%; Average loss: 4.9355
Iteration: 548; Percent complete: 13.7%; Average loss: 5.0315
Iteration: 549; Percent complete: 13.7%; Average loss: 4.8484
Iteration: 550; Percent complete: 13.8%; Average loss: 4.8632
Iteration: 551; Percent complete: 13.8%; Average loss: 4.9235
Iteration: 552; Percent complete: 13.8%; Average loss: 4.9188
Iteration: 553; Percent complete: 13.8%; Average loss: 4.9325
Iteration: 554; Percent complete: 13.9%; Average loss: 4.9650
Iteration: 555; Percent complete: 13.9%; Average loss: 4.9445
Iteration: 556; Percent complete: 13.9%; Average loss: 4.9299
Iteratio

...

Iteration: 3447; Percent complete: 86.2%; Average loss: 4.1905
Iteration: 3448; Percent complete: 86.2%; Average loss: 4.0945
Iteration: 3449; Percent complete: 86.2%; Average loss: 3.9455
Iteration: 3450; Percent complete: 86.2%; Average loss: 4.0884
Iteration: 3451; Percent complete: 86.3%; Average loss: 4.0147
Iteration: 3452; Percent complete: 86.3%; Average loss: 4.2100
Iteration: 3453; Percent complete: 86.3%; Average loss: 4.0564
Iteration: 3454; Percent complete: 86.4%; Average loss: 4.2417
Iteration: 3455; Percent complete: 86.4%; Average loss: 4.0683
Iteration: 3456; Percent complete: 86.4%; Average loss: 4.1204
Iteration: 3457; Percent complete: 86.4%; Average loss: 4.1313
Iteration: 3458; Percent complete: 86.5%; Average loss: 4.1368
Iteration: 3459; Percent complete: 86.5%; Average loss: 4.0501
Iteration: 3460; Percent complete: 86.5%; Average loss: 4.1055
Iteration: 3461; Percent complete: 86.5%; Average loss: 4.0846
Iteration: 3462; Percent complete: 86.6%; Average loss:

Iteration: 3579; Percent complete: 89.5%; Average loss: 4.1451
Iteration: 3580; Percent complete: 89.5%; Average loss: 3.9982
Iteration: 3581; Percent complete: 89.5%; Average loss: 3.9884
Iteration: 3582; Percent complete: 89.5%; Average loss: 4.0701
Iteration: 3583; Percent complete: 89.6%; Average loss: 4.1024
Iteration: 3584; Percent complete: 89.6%; Average loss: 4.0763
Iteration: 3585; Percent complete: 89.6%; Average loss: 4.1237
Iteration: 3586; Percent complete: 89.6%; Average loss: 4.0016
Iteration: 3587; Percent complete: 89.7%; Average loss: 4.1142
Iteration: 3588; Percent complete: 89.7%; Average loss: 3.9885
Iteration: 3589; Percent complete: 89.7%; Average loss: 4.0558
Iteration: 3590; Percent complete: 89.8%; Average loss: 4.1102
Iteration: 3591; Percent complete: 89.8%; Average loss: 3.9811
Iteration: 3592; Percent complete: 89.8%; Average loss: 4.1094
Iteration: 3593; Percent complete: 89.8%; Average loss: 3.9893
Iteration: 3594; Percent complete: 89.8%; Average loss:

Iteration: 3710; Percent complete: 92.8%; Average loss: 3.9648
Iteration: 3711; Percent complete: 92.8%; Average loss: 4.1773
Iteration: 3712; Percent complete: 92.8%; Average loss: 4.1557
Iteration: 3713; Percent complete: 92.8%; Average loss: 4.0662
Iteration: 3714; Percent complete: 92.8%; Average loss: 4.0172
Iteration: 3715; Percent complete: 92.9%; Average loss: 4.0448
Iteration: 3716; Percent complete: 92.9%; Average loss: 3.9319
Iteration: 3717; Percent complete: 92.9%; Average loss: 4.1388
Iteration: 3718; Percent complete: 93.0%; Average loss: 4.0119
Iteration: 3719; Percent complete: 93.0%; Average loss: 4.0367
Iteration: 3720; Percent complete: 93.0%; Average loss: 3.9867
Iteration: 3721; Percent complete: 93.0%; Average loss: 4.1065
Iteration: 3722; Percent complete: 93.0%; Average loss: 4.0448
Iteration: 3723; Percent complete: 93.1%; Average loss: 4.0964
Iteration: 3724; Percent complete: 93.1%; Average loss: 3.9996
Iteration: 3725; Percent complete: 93.1%; Average loss:

Iteration: 3841; Percent complete: 96.0%; Average loss: 4.1222
Iteration: 3842; Percent complete: 96.0%; Average loss: 3.8147
Iteration: 3843; Percent complete: 96.1%; Average loss: 4.1110
Iteration: 3844; Percent complete: 96.1%; Average loss: 3.9865
Iteration: 3845; Percent complete: 96.1%; Average loss: 4.1468
Iteration: 3846; Percent complete: 96.2%; Average loss: 3.8770
Iteration: 3847; Percent complete: 96.2%; Average loss: 4.0535
Iteration: 3848; Percent complete: 96.2%; Average loss: 4.0726
Iteration: 3849; Percent complete: 96.2%; Average loss: 4.0710
Iteration: 3850; Percent complete: 96.2%; Average loss: 4.0553
Iteration: 3851; Percent complete: 96.3%; Average loss: 3.9971
Iteration: 3852; Percent complete: 96.3%; Average loss: 4.0039
Iteration: 3853; Percent complete: 96.3%; Average loss: 4.0101
Iteration: 3854; Percent complete: 96.4%; Average loss: 3.9279
Iteration: 3855; Percent complete: 96.4%; Average loss: 4.1013
Iteration: 3856; Percent complete: 96.4%; Average loss:

Iteration: 3972; Percent complete: 99.3%; Average loss: 3.9216
Iteration: 3973; Percent complete: 99.3%; Average loss: 3.9747
Iteration: 3974; Percent complete: 99.4%; Average loss: 3.9043
Iteration: 3975; Percent complete: 99.4%; Average loss: 4.0095
Iteration: 3976; Percent complete: 99.4%; Average loss: 3.9373
Iteration: 3977; Percent complete: 99.4%; Average loss: 3.9951
Iteration: 3978; Percent complete: 99.5%; Average loss: 4.0611
Iteration: 3979; Percent complete: 99.5%; Average loss: 3.9300
Iteration: 3980; Percent complete: 99.5%; Average loss: 4.0072
Iteration: 3981; Percent complete: 99.5%; Average loss: 3.8594
Iteration: 3982; Percent complete: 99.6%; Average loss: 4.0378
Iteration: 3983; Percent complete: 99.6%; Average loss: 4.0189
Iteration: 3984; Percent complete: 99.6%; Average loss: 3.9769
Iteration: 3985; Percent complete: 99.6%; Average loss: 4.0377
Iteration: 3986; Percent complete: 99.7%; Average loss: 3.9891
Iteration: 3987; Percent complete: 99.7%; Average loss:

In [23]:
torch.save(encoder.state_dict(), 'data/save/cb_model/kaggle-encoder.pt')
torch.save(decoder.state_dict(), 'data/save/cb_model/kaggle-decoder.pt')


### Run Evaluation

To chat with your model, run the following block.




In [21]:
# Set dropout layers to ``eval`` mode
encoder.eval()
decoder.eval()

# Initialize search module
searcher = GreedySearchDecoder(encoder, decoder)

In [ ]:
# Begin chatting (uncomment and run the following line to begin)
evaluateInput(encoder, decoder, searcher, voc)

In [33]:
#convert encoder

In [152]:
indexes_batch = [[1925, 2, 3]]

lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
# Transpose dimensions of batch to match models' expectations
print(indexes_batch)
input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
# Use appropriate device
input_batch = input_batch.to('cpu')
lengths = lengths.to("cpu")

[[1925, 2, 3]]


In [153]:
input_batch.shape

torch.Size([3, 1])

In [154]:
lengths.shape

torch.Size([1])

In [158]:
# Input to the model
x = torch.randn(2, 1, requires_grad=True)

# Export the model
torch.onnx.export(encoder.cpu(),              
  (input_batch, lengths),                         # model input (or a tuple for multiple inputs)
  "encoder.onnx",            # where to save the model (can be a file or file-like object)
  export_params=True,        # store the trained parameter weights inside the model file
  opset_version=18,          # the ONNX version to export the model to
  do_constant_folding=True,  # whether to execute constant folding for optimization
  input_names = ('inputs', 'input_length'),   # the model's input names
  output_names = ('encoder_outputs', 'encoder_hidden'), # the model's output names
  dynamic_axes={'input_seq' : {0 : 'batch'},    # variable length axes
                'input_length' : {0 : 'batch'}})

/usr/local/lib/python3.10/dist-packages/torch/onnx/utils.py:2046: UserWarning: Provided key input_seq for dynamic axes is not a valid input/output name
  warnings.warn(
[W shape_type_inference.cpp:1956] Warning: The shape inference of prim::PackPadded type is missing, so it may result in wrong shape inference for the exported graph. Please consider adding it in symbolic function. (function UpdateReliable)
The shape inference of prim::PackPadded type is missing, so it may result in wrong shape inference for the exported graph. Please consider adding it in symbolic function.
None
The shape inference of prim::PackPadded type is missing, so it may result in wrong shape inference for the exported graph. Please consider adding it in symbolic function.
None
[W shape_type_inference.cpp:1956] Warning: The shape inference of prim::PackPadded type is missing, so it may result in wrong shape inference for the exported graph. Please consider adding it in symbolic function. (function UpdateReliable)

=========== Diagnostic Run torch.onnx.export version 2.1.0a0+b5021ba ===========
verbose: False, log level: 40
======================= 0 NONE 0 NOTE 4 WARNING 0 ERROR ========================
4 WARNING were not printed due to the log level.



In [98]:
import onnx
encoder_onnx = onnx.load('encoder.onnx')

In [99]:
encoder_onnx.graph.input

[name: "input_seq"
type {
  tensor_type {
    elem_type: 7
    shape {
      dim {
        dim_value: 2
      }
      dim {
        dim_value: 1
      }
    }
  }
}
, name: "input_length"
type {
  tensor_type {
    elem_type: 7
    shape {
      dim {
        dim_value: 1
      }
    }
  }
}
]

In [118]:
torch_out = encoder(input_batch, lengths)

In [39]:
torch_out[1]

tensor([[[ 0.2754,  0.0493,  0.2588,  ..., -0.2692, -0.2900,  0.2154]],

        [[ 0.2009, -0.3519, -0.4728,  ..., -0.3503,  0.2891,  0.5329]],

        [[ 0.2038, -0.1514,  0.1179,  ...,  0.2450, -0.2199,  0.0358]],

        [[-0.4824,  0.4065,  0.0157,  ..., -0.1548, -0.5504, -0.3053]]],
       device='cuda:0', grad_fn=<CudnnRnnBackward0>)

In [41]:
!pip install onnxruntime

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 26.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 148.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 180.1 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip


In [116]:
import onnxruntime
import numpy as np
ort_session = onnxruntime.InferenceSession("encoder.onnx")

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# compute ONNX Runtime output prediction
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(input_batch), ort_session.get_inputs()[1].name : to_numpy(lengths)}
print(ort_inputs )
ort_outs = ort_session.run(None, ort_inputs)

# # compare ONNX Runtime and PyTorch results
# np.testing.assert_allclose(to_numpy(torch_out[1]), ort_outs[1], rtol=1e-03, atol=1e-05)

# print("Exported model has been tested with ONNXRuntime, and the result looks good!")

{'input_seq': array([[1925],
       [   2]]), 'input_length': array([2])}


In [119]:
print((to_numpy(torch_out[0]) - ort_outs[0]).mean())
print((to_numpy(torch_out[1]) - ort_outs[1]).mean())

-2.3829052e-09
8.072821e-10


In [34]:
#convert decoder

In [120]:
decoder_input = torch.ones(1, 1, dtype=torch.long) * 1 *SOS_token
encoder_outputs = torch_out[0].cpu()
decoder_hidden = torch_out[1][:2].cpu()

In [124]:
print('decoder_input', decoder_input.shape)
print('decoder_hidden', decoder_hidden.shape)
print('encoder_outputs', encoder_outputs.shape)

decoder_input torch.Size([1, 1])
decoder_hidden torch.Size([2, 1, 500])
encoder_outputs torch.Size([2, 1, 500])


In [134]:
# Input to the model
x = torch.randn(2, 1, requires_grad=True)

# Export the model
torch.onnx.export(decoder.cpu(),              
  (decoder_input, decoder_hidden, encoder_outputs),                         # model input (or a tuple for multiple inputs)
  "decoder.onnx",            # where to save the model (can be a file or file-like object)
  export_params=True,        # store the trained parameter weights inside the model file
  opset_version=18,          # the ONNX version to export the model to
  do_constant_folding=True,  # whether to execute constant folding for optimization
  input_names = ('input_step', 'last_hidden', 'encoder_outputs'),  # the model's input names
  output_names = ('decoder_outputs', 'decoder_hidden'), # the model's output names
  dynamic_axes={  
                'last_hidden' : {0 : 'batch'},
                'encoder_outputs' : {0 : 'batch'}})

=========== Diagnostic Run torch.onnx.export version 2.1.0a0+b5021ba ===========
verbose: False, log level: 40
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [130]:
decoder_ort_session = onnxruntime.InferenceSession("decoder.onnx")

# compute ONNX Runtime output prediction
decoder_ort_inputs = {
    decoder_ort_session.get_inputs()[0].name: to_numpy(decoder_input), 
    decoder_ort_session.get_inputs()[1].name : to_numpy(decoder_hidden),
    decoder_ort_session.get_inputs()[2].name : to_numpy(encoder_outputs),
}
decoder_ort_outs = decoder_ort_session.run(None, decoder_ort_inputs)


In [131]:
decoder_torch_out = decoder(decoder_input, decoder_hidden, encoder_outputs)

In [132]:
print((to_numpy(decoder_torch_out [0]) - decoder_ort_outs[0]).mean())
print((to_numpy(decoder_torch_out [1]) - decoder_ort_outs[1]).mean())

-5.5246387e-11
3.746245e-10


In [82]:
print((to_numpy(decoder_torch_out [0]) - decoder_ort_outs[0]).max())
print((to_numpy(decoder_torch_out [1]) - decoder_ort_outs[1]).max())

2.561137e-09
2.9802322e-07
